# ETL: Limpieza y Normalización
Este notebook integra la carga de datos (y las variables de rutas) importando el notebook `cargar_y_columnas.ipynb`, ejecuta la normalización de fechas y la vista previa del resultado final.

In [10]:
import pandas as pd
import pathlib
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
# Importar modulos externos
# Al correr esto, se cargarán las funciones y las variables
%run cargar_y_columnas.ipynb
# Llamamos al modulo de normalización de fecha
%run normalizacion_fecha.ipynb
# Llamamos al modulo de normalización de departamento
%run normalizacion_departamento.ipynb

## Verificación de Carga y Columnas

In [11]:
dfs_cargados = {}

for tipo, ruta in archivos.items():
    # Carga que invoca la función importada en el %run cargar_y_columnas.ipynb
    df = cargar_y_normalizar_columnas(ruta)
    
    if df is not None:
        dfs_cargados[tipo] = df
        display(df.head(1))

,recaudador,cuota,intereses,canal,departamento,mesrecaudo
0,COMERCIALIZADORA JFS PAPAS S.A.S.,65300.7851,0,CENTRAL DE ABASTOS,"Bogotá, D.C.",Septiembre - 2024


,ciudad,departamento,año,mes,semana,variedad,departamento2,fecha3,precio,longitud,latitud
0,El Carmen de Viboral,Antioquia,2012,abril,15,Criolla Limpia,Antioquia,2012-04-08,539.577292,"-75,335269012","6,08497157368"


,año,mes,semestre,departamento,produccion_ton_
0,2024,1,I,ANTIOQUIA,13366.50454


## ¿Existen valores nulos explicitos en el conjunto de datos?

In [15]:
for tipo, df_crudo in dfs_cargados.items():
    print(f'--- Nulos en {tipo.upper()} ---')
    display(
        df_crudo
        .isnull()
        .sum()
        .sort_values(ascending=False)
    )
    print('\n')

--- Nulos en RECAUDO ---


recaudador      0
cuota           0
intereses       0
canal           0
departamento    0
mesrecaudo      0
dtype: int64



--- Nulos en PRECIOS ---


longitud         10299
latitud          10299
ciudad               0
departamento         0
año                  0
mes                  0
semana               0
variedad             0
departamento2        0
fecha3               0
precio               0
dtype: int64



--- Nulos en PRODUCCION ---


año                0
mes                0
semestre           0
departamento       0
produccion_ton_    0
dtype: int64

## Ejecución Principal con Funciones Adicionales

In [7]:
dfs_procesados = {}

# Usamos los dataframes que ya cargamos y verificamos en la celda anterior
for tipo, df_crudo in dfs_cargados.items():
    print(f"-- APLICANDO NORMALIZACIÓN FECHA Y DEPARTAMENTO: {tipo.upper()} --")
    
    # Función normalizar_fecha invocada desde %run normalizacion_fecha.ipynb
    df_transformado = normalizar_fecha(df_crudo, tipo)
    
    # Función normalizar_departamentos invocada desde %run normalizacion_departamento.ipynb
    df_transformado = normalizar_departamentos(df_transformado, tipo)
    
    dfs_procesados[tipo] = df_transformado
    display(df_transformado.head(1))

-- APLICANDO NORMALIZACIÓN FECHA Y DEPARTAMENTO: RECAUDO --


,recaudador,cuota,intereses,canal,departamento,mesrecaudo,mes,año,fecha_normalizada,departamento_normalizado
0,COMERCIALIZADORA JFS PAPAS S.A.S.,65300.7851,0,CENTRAL DE ABASTOS,"Bogota, D.C.",Septiembre - 2024,9,2024,2024-09-01,Cundinamarca


-- APLICANDO NORMALIZACIÓN FECHA Y DEPARTAMENTO: PRECIOS --


,ciudad,departamento,año,mes,semana,variedad,departamento2,fecha3,precio,longitud,latitud,fecha_normalizada,departamento_normalizado
0,El Carmen de Viboral,Antioquia,2012,abril,15,Criolla Limpia,Antioquia,2012-04-08,539.577292,"-75,335269012","6,08497157368",2012-04-01,Antioquia


-- APLICANDO NORMALIZACIÓN FECHA Y DEPARTAMENTO: PRODUCCION --


,año,mes,semestre,departamento,produccion_ton_,fecha_normalizada,departamento_normalizado
0,2024,1,I,Antioquia,13366.50454,2024-01-01,Antioquia


In [9]:
# Guardar localmente
import os

os.makedirs("data_lista_para_subir", exist_ok=True)
for nombre, df in dfs_procesados.items():
    ruta = f"data_lista_para_subir/{nombre}.csv"
    df.to_csv(ruta, index=False)
    print(f"Guardado localmente: {ruta}")


Guardado localmente: data_lista_para_subir/recaudo.csv
Guardado localmente: data_lista_para_subir/precios.csv
Guardado localmente: data_lista_para_subir/produccion.csv
